In [1]:
import pandas as pd


# Loading datasets
higher_ed_df = pd.read_csv('dataset/subset_higher_ed_ednum.csv')
some_college_df = pd.read_csv('dataset/subset_some_college_ednum.csv')
less_than_hs_df = pd.read_csv('dataset/subset_less_than_hs_ednum.csv')

In [2]:
from sklearn.preprocessing import LabelEncoder


# Dropping 'fnlgwt' and 'Education' features from all datasets as they are not useful or duplicated
higher_ed_df = higher_ed_df.drop(['fnlgwt', 'Education'], axis=1)
some_college_df = some_college_df.drop(['fnlgwt', 'Education'], axis=1)
less_than_hs_df = less_than_hs_df.drop(['fnlgwt', 'Education'], axis=1)

# Encoding categorical features in all datasets
def encode_features(df):
    label_encoders = {}
    for column in df.select_dtypes(include=['object']).columns:
        label_encoders[column] = LabelEncoder()
        df[column] = label_encoders[column].fit_transform(df[column])
    return df, label_encoders

higher_ed_df_encoded, higher_ed_encoders = encode_features(higher_ed_df.copy())
some_college_df_encoded, some_college_encoders = encode_features(some_college_df.copy())
less_than_hs_df_encoded, less_than_hs_encoders = encode_features(less_than_hs_df.copy())

(higher_ed_df_encoded.head(), some_college_df_encoded.head(), less_than_hs_df_encoded.head())

(   Age  Workclass  EdNum  MaritalStatus  Occupation  Relationship  Race  Sex  \
 0   39          5     13              4           0             1     4    1   
 1   50          4     13              2           3             0     4    1   
 2   28          2     13              2           9             5     2    0   
 3   37          2     14              2           3             5     4    0   
 4   31          2     14              4           9             1     4    0   
 
    CapitalGain  CapitalLoss  HoursPerWeek  Country  Income  
 0         2174            0            40       37       0  
 1            0            0            13       37       0  
 2            0            0            40        4       0  
 3            0            0            40       37       0  
 4        14084            0            50       37       1  ,
    Age  Workclass  EdNum  MaritalStatus  Occupation  Relationship  Race  Sex  \
 0   38          2      9              0           5      

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
import matplotlib.pyplot as plt


# Defining the input features and target variable for each subset
X_higher_ed = higher_ed_df_encoded.drop('Income', axis=1)
y_higher_ed = higher_ed_df_encoded['Income']

X_some_college = some_college_df_encoded.drop('Income', axis=1)
y_some_college = some_college_df_encoded['Income']

X_less_than_hs = less_than_hs_df_encoded.drop('Income', axis=1)
y_less_than_hs = less_than_hs_df_encoded['Income']

1. K-Nearest Neighbors (KNN)

Function Call:
~~~python
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=5)
~~~
Hyperparameter: n_neighbors

Values for Testing: 1, 3, 5, 7, 9

In [ ]:
# Setting up K values for tuning
k_values = [1, 3, 5, 7, 9]

# Function to perform KNN tuning and plotting
def tune_knn(X, y, k_values, title):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    mean_cv_scores = []
    for k in k_values:
        knn_clf = KNeighborsClassifier(n_neighbors=k)
        cv_scores = cross_val_score(knn_clf, X, y, cv=5)
        mean_cv_scores.append(np.mean(cv_scores))
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(k_values, mean_cv_scores, marker='o')
    plt.title(f'5-Fold CV Accuracy vs. K for {title}')
    plt.xlabel('Number of Neighbors (k)')
    plt.ylabel('CV Mean Accuracy')
    plt.grid(True)
    plt.show()

tune_knn(X_higher_ed, y_higher_ed, k_values, "Higher Education Subset")
tune_knn(X_higher_ed, y_higher_ed, k_values, "Some College Subset")
tune_knn(X_higher_ed, y_higher_ed, k_values, "Less than High School Subset")


2. Artificial Neural Network (ANN)

Function Call:
~~~python
from sklearn.neural_network import MLPClassifier
ann_clf = MLPClassifier(hidden_layer_sizes=(100,), activation='relu')
~~~
Hyperparameter: hidden_layer_sizes (for the number of neurons in the hidden layers)

Values for Testing: [(50,), (100,), (100,50), (100,100), (50,50,50)]

In [ ]:
from sklearn.neural_network import MLPClassifier


# Configurations for the hidden layer sizes to test
hidden_layer_sizes = [(50,), (100,), (100,50), (100,100), (50,50,50)]

def tune_ann(X, y, hidden_layer_sizes, title):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    mean_cv_scores_ann = []
    for size in hidden_layer_sizes:
        print('Size of hidden layer: ', size)
        ann_clf = MLPClassifier(hidden_layer_sizes=size, activation='relu', max_iter=1000)
        cv_scores = cross_val_score(ann_clf, X, y, cv=5)
        mean_cv_scores_ann.append(np.mean(cv_scores))
    
    plt.figure(figsize=(10, 6))
    plt.plot([str(size) for size in hidden_layer_sizes], mean_cv_scores_ann, marker='o')
    plt.title(f'5-Fold CV Accuracy vs. Hidden Layer Sizes for {title}')
    plt.xlabel('Number of Neurons in Hidden Layer')
    plt.ylabel('CV Mean Accuracy')
    plt.grid(True)
    plt.show()


tune_ann(X_higher_ed, y_higher_ed, hidden_layer_sizes, "Higher Education Subset")
tune_ann(X_higher_ed, y_higher_ed, hidden_layer_sizes, "Some College Subset")
tune_ann(X_higher_ed, y_higher_ed, hidden_layer_sizes, "Less than High School Subset")

In [4]:
import torch
import torch.nn as nn
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, TensorDataset, Subset

class CustomANN(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size):
        super(CustomANN, self).__init__()
        layers = [nn.Linear(input_size, hidden_layers[0]), nn.ReLU()]
        
        for i in range(len(hidden_layers) - 1):
            layers.append(nn.Linear(hidden_layers[i], hidden_layers[i + 1]))
            layers.append(nn.ReLU())
        
        layers.append(nn.Linear(hidden_layers[-1], output_size))
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.net(x)


In [5]:
def tune_ann_pytorch(X, y, hidden_layer_sizes, title, k_folds=5, epochs=100):
    print(1)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)
    print(2)
    dataset = TensorDataset(X_tensor, y_tensor)
    
    kfold = KFold(n_splits=k_folds, shuffle=True)
    
    results = {}
    
    for size in hidden_layer_sizes:
        fold_performance = []
        
        for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
            print(f'Fold {fold}, Hidden Layer Size: {size}')
            
            train_subsampler = Subset(dataset, train_ids)
            test_subsampler = Subset(dataset, test_ids)
            
            train_loader = DataLoader(train_subsampler, batch_size=64, shuffle=True)
            test_loader = DataLoader(test_subsampler, batch_size=64, shuffle=False)
            
            model = CustomANN(input_size=X.shape[1], hidden_layers=size, output_size=len(np.unique(y)))
            model.to('cuda' if torch.cuda.is_available() else 'cpu')
            criterion = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            
            # Training loop
            for epoch in range(epochs):
                model.train()
                for data, target in train_loader:
                    data, target = data.to('cuda'), target.to('cuda')
                    optimizer.zero_grad()
                    output = model(data)
                    loss = criterion(output, target)
                    loss.backward()
                    optimizer.step()
            
            # Evaluation loop
            model.eval()
            correct = 0
            with torch.no_grad():
                for data, target in test_loader:
                    data, target = data.to('cuda'), target.to('cuda')
                    output = model(data)
                    pred = output.argmax(dim=1, keepdim=True)
                    correct += pred.eq(target.view_as(pred)).sum().item()
            
            fold_performance.append(100. * correct / len(test_subsampler))
        
        results[size] = np.mean(fold_performance)
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot([str(size) for size in hidden_layer_sizes], list(results.values()), marker='o')
    plt.title(f'5-Fold CV Accuracy vs. Hidden Layer Sizes for {title}')
    plt.xlabel('Number of Neurons in Hidden Layer(s)')
    plt.ylabel('CV Mean Accuracy (%)')
    plt.grid(True)
    plt.show()


In [ ]:
hidden_layer_sizes = [(50,), (100,), (100,50), (100,100), (50,50,50)]

# 调用函数评估“Higher Education Subset”
tune_ann_pytorch(X_higher_ed, y_higher_ed, hidden_layer_sizes, "Higher Education Subset")

3. Decision Tree

Function Call:

~~~python
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(max_depth=5)
~~~
Hyperparameter: max_depth

Values for Testing: 3, 5, 7, 9, None (no limit)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

    
def tune_decision_tree_actual_depth(X, y, max_depth_values, title):
    mean_cv_scores_dt = []
    depths_for_plot = max_depth_values.copy()  # Copy the list to avoid modifying the original
    
    for depth in max_depth_values:
        if depth is None:
            # Fit the tree without depth restriction
            dt_clf = DecisionTreeClassifier(max_depth=None)
            dt_clf.fit(X, y)  # Fit the model to find out the actual depth
            actual_depth = dt_clf.get_depth()  # Get the actual depth of the tree
            depths_for_plot[-1] = actual_depth  # Replace None with actual depth for plotting
        # else:
        dt_clf = DecisionTreeClassifier(max_depth=depth)
        
        cv_scores = cross_val_score(dt_clf, X, y, cv=5)
        mean_cv_scores_dt.append(np.mean(cv_scores))
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(depths_for_plot, mean_cv_scores_dt, marker='o')
    plt.title(f'5-Fold CV Accuracy vs. Max Depth for {title} (Actual Depth for None)')
    plt.xlabel('Max Depth')
    plt.ylabel('CV Mean Accuracy')
    plt.grid(True)
    plt.xticks(depths_for_plot)
    plt.show()


# Define the list of max_depth values to test
max_depth_values = [3, 5, 7, 9, None]

tune_decision_tree_actual_depth(X_higher_ed, y_higher_ed, max_depth_values, "Higher Education Subset")
tune_decision_tree_actual_depth(X_some_college, y_some_college, max_depth_values, "Some College Subset")
tune_decision_tree_actual_depth(X_less_than_hs, y_less_than_hs, max_depth_values, "Less than High School Subset")

4. Random Forest

Function Call:

~~~python
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=100)
~~~
Hyperparameter: n_estimators (number of trees)

Values for Testing: 10, 50, 100, 150, 200

In [ ]:
from sklearn.ensemble import RandomForestClassifier


def tune_random_forest(X, y, n_estimators_values, title):
    mean_cv_scores_rf = []
    
    for n_estimators in n_estimators_values:
        rf_clf = RandomForestClassifier(n_estimators=n_estimators)
        cv_scores = cross_val_score(rf_clf, X, y, cv=5)
        mean_cv_scores_rf.append(np.mean(cv_scores))
    
    plt.figure(figsize=(10, 6))
    plt.plot(n_estimators_values, mean_cv_scores_rf, marker='o')
    plt.title(f'5-Fold CV Accuracy vs. Number of Trees for {title}')
    plt.xlabel('Number of Trees (n_estimators)')
    plt.ylabel('CV Mean Accuracy')
    plt.grid(True)
    plt.show()

n_estimators_values = [10, 50, 100, 150, 200]

tune_random_forest(X_higher_ed, y_higher_ed, n_estimators_values, "Higher Education Subset")
tune_random_forest(X_some_college, y_some_college, n_estimators_values, "Some College Subset")
tune_random_forest(X_less_than_hs, y_less_than_hs, n_estimators_values, "Less than High School Subset")

5. Naïve Bayes Classifier

Function Call:
~~~python
from sklearn.naive_bayes import GaussianNB
nb_clf = GaussianNB(var_smoothing=1e-9)
~~~
Hyperparameter: var_smoothing (portion of the largest variance of all features added to variances for calculation stability)

Values for Testing: 1e-9, 1e-8, 1e-7, 1e-6, 1e-5

In [ ]:
from sklearn.naive_bayes import GaussianNB


def tune_naive_bayes(X, y, var_smoothing_values, title):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    mean_cv_scores_nb = []
    
    for var_smoothing in var_smoothing_values:
        nb_clf = GaussianNB(var_smoothing=var_smoothing)
        cv_scores = cross_val_score(nb_clf, X, y, cv=5)
        mean_cv_scores_nb.append(np.mean(cv_scores))
    
    plt.figure(figsize=(10, 6))
    plt.semilogx(var_smoothing_values, mean_cv_scores_nb, marker='o', base=10)
    plt.title(f'5-Fold CV Accuracy vs. Var Smoothing for {title}')
    plt.xlabel('Var Smoothing')
    plt.ylabel('CV Mean Accuracy')
    plt.grid(True)
    plt.show()

var_smoothing_values = [1e-12, 1e-9, 1e-6, 1e-3, 1e-1]

tune_naive_bayes(X_higher_ed, y_higher_ed, var_smoothing_values, "Higher Education Subset")
tune_naive_bayes(X_some_college, y_some_college, var_smoothing_values, "Some College Subset")
tune_naive_bayes(X_less_than_hs, y_less_than_hs, var_smoothing_values, "Less than High School Subset")

6. Support Vector Machine (SVM)

Function Call:
~~~python
from sklearn.svm import SVC
svm_clf = SVC(kernel='rbf')
~~~
Hyperparameters: kernel

Values for Testing: ['rbf', 'sigmoid', 'linear', 'poly']

In [ ]:
from sklearn.svm import SVC, LinearSVC


def tune_svm(X, y, kernels, title):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    mean_cv_scores_svm = []

    for kernel in kernels:
        print("Processing:", kernel)
        if kernel == 'linear':
            # 使用LinearSVC
            svm_clf = LinearSVC()
        else:
            # 使用SVC的其他核
            svm_clf = SVC(kernel=kernel)

        cv_scores = cross_val_score(svm_clf, X, y, cv=5)
        mean_cv_scores_svm.append(np.mean(cv_scores))

    # 可视化结果
    plt.figure(figsize=(10, 6))
    plt.bar(kernels, mean_cv_scores_svm)
    plt.title(f'5-Fold CV Accuracy vs. Kernel for {title}')
    plt.xlabel('Kernel')
    plt.ylabel('CV Mean Accuracy')
    plt.grid(True, axis='y')
    plt.show()

kernels = ['rbf', 'sigmoid', 'linear', 'poly']

tune_svm(X_higher_ed, y_higher_ed, kernels, "Higher Education Subset")
tune_svm(X_some_college, y_some_college, kernels, "Some College Subset")
tune_svm(X_less_than_hs, y_less_than_hs, kernels, "Less than High School Subset")